this script calculate the unit flow between each country pair

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from ast import literal_eval
from collections import Counter
from itertools import product

In [2]:
pubs_path="../../data/nf_folder/Data/DerivedData/CleanedRawData/pubs_author_funder.csv"

In [3]:
flow_output="../../data/nf_folder/Data/DerivedData/Derived/funding_source_target.csv"

In [4]:
pubs_author_funder = pd.read_csv(pubs_path)
pubs_author_funder['author_country'] = pubs_author_funder['author_country'].apply(literal_eval)
pubs_author_funder['funder_country'] = pubs_author_funder['funder_country'].apply(
    lambda x:literal_eval(x) if pd.notnull(x) else "Not-Funded")
pubs_author_funder.head()

/home/miaoli/conda/envs/funding/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,author_country,funder_country,discipline,year,rel_citation,citation
0,30507292,[Turkey],[Turkey],77,2009,0.485,6
1,30574690,"[United States, France]","[United States, United States, United States, ...",83,2009,0.703,9
2,30574691,[Russia],[Russia],83,2009,0.312,4
3,30585095,"[Spain, Spain]","[Spain, Spain, Spain, Spain, Spain, Spain]",89,2009,0.137,1
4,30585097,"[Germany, Spain]",[Spain],89,2009,0.410,3


In [5]:
pubs_author_funder['author_distinct']=pubs_author_funder['author_country'].apply(lambda x:list(set(x)))
pubs_author_funder['funder_distinct']=pubs_author_funder['funder_country'].apply(
    lambda x:list(set(x)) if x!='Not-Funded' else 'Not-Funded')

In [12]:
pubs_author_funder['coll']=pubs_author_funder['author_distinct'].apply(
    lambda x: 1 if len(x)>1 else 0)
pubs_author_funder['fund']=pubs_author_funder['funder_distinct'].apply(
    lambda x: 1 if x!='Not-Funded' else 0)

In [14]:
pubs_author_funder.tail()

,id,author_country,funder_country,discipline,year,rel_citation,citation,author_distinct,funder_distinct,coll,fund
12759123,65301180,[Pakistan],Not-Funded,89,2018,0.000,0,[Pakistan],Not-Funded,0,0
12759124,65301286,"[Singapore, Singapore]",Not-Funded,44,2017,0.264,1,[Singapore],Not-Funded,0,0
12759125,65301288,[Iran],Not-Funded,96,2017,0.000,0,[Iran],Not-Funded,0,0
12759126,65301295,[Israel],Not-Funded,96,2017,0.000,0,[Israel],Not-Funded,0,0
12759127,65301297,[United Kingdom],Not-Funded,2,2018,1.992,2,[United Kingdom],Not-Funded,0,0


In [15]:
fracaut=[]
for ind, row in pubs_author_funder.iterrows():
    authors=row['author_distinct']
    n=len(authors)
    coll=row['coll']
    year=row['year']
    fund=row['fund']
    for a in authors:
        fracaut.append([year,a, 1/n, coll,fund])

In [16]:
frac_df=pd.DataFrame(fracaut, columns=['year','cntry','unit','coll','fund'])
frac_df.head()

,year,cntry,unit,coll,fund
0,2009,Turkey,1.0,0,1
1,2009,France,0.5,1,1
2,2009,United States,0.5,1,1
3,2009,Russia,1.0,0,1
4,2009,Spain,1.0,0,1


In [17]:
all=frac_df.groupby(['year','cntry'])['unit'].sum().reset_index()
all.head()

,year,cntry,unit
0,2009,Afghanistan,8.750000
1,2009,Albania,40.416667
2,2009,Algeria,1061.854652
3,2009,Andorra,1.500000
4,2009,Angola,12.530409


In [24]:
all[(all.year==2016)&(all.cntry=='United States')]

,year,cntry,unit
1592,2016,United States,255835.38531


In [25]:
flowresult=[]
pubs_filter=pubs_author_funder[pubs_author_funder.funder_country!='Not-Funded']
for ind, row in pubs_filter.iterrows():
    authordict=Counter(row['author_country'])
    funderdict=Counter(row['funder_country'])
    num_aut=sum(authordict.values())
    for a, f in product(authordict.keys(),funderdict.keys()):
        f_unit=funderdict[f]
        a_p=authordict[a]
        value=(a_p/num_aut)*f_unit
        flowresult.append([f,a,value])

In [32]:
flow_df=pd.DataFrame(flowresult, columns=['source','target','value'])
flow_df=flow_df.groupby(['source','target'])['value'].sum().reset_index()
flow_df

,source,target,value
0,Afghanistan,Afghanistan,0.500000
1,Afghanistan,Australia,0.333333
2,Afghanistan,Bangladesh,0.666667
3,Afghanistan,Bolivia,0.125000
4,Afghanistan,Canada,1.132576
...,...,...,...
14144,Zimbabwe,United Kingdom,9.802381
14145,Zimbabwe,United States,23.297150
14146,Zimbabwe,Vietnam,0.500000
14147,Zimbabwe,Zambia,2.242857


In [34]:
flow_df.to_csv(flow_output, index=False)